<insert. Fannie Mae logo>

___________
# Mortgage Loan Default Classifier

### Problem Statement:
- Predict whether a mortgage loan will default using Fannie Mae Loan Performance data set from previous quarter

-----------

## EDA and Cleaning
___

In [1]:
# import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# define function to to allow user to assign train year and quarter of training data

In [8]:
# define function to load and clean data set

def load_dataset(year=2005, quarter=1):

    #load acquisition data dictionary csv for column names
    acq_data_dict_fp = './data/acquisition_data_dict_summary.csv'
    list_acq_col_names = pd.read_csv(acq_data_dict_fp)['Field Name'].tolist()

    #load acquisition data dictionary csv for column names
    perf_data_dict_fp = './data/performance_data_dict_summary.csv'
    list_perf_col_names = pd.read_csv(perf_data_dict_fp)['Field Name'].tolist()

    #load acquisition csv
    ## string for fp needs variable
    acq_data_fp = f'./data/2005Q1/Acquisition_{year}Q{quarter}.txt'
    df_acq = pd.read_csv(acq_data_fp, sep='|', header=None, names=list_acq_col_names)
    
    #load performance csv
    perf_data_fp = f'./data/2005Q1/Performance_{year}Q{quarter}.txt'
    df_perf = pd.read_csv(perf_data_fp, sep='|', header=None, names=list_perf_col_names)
    
    return df_acq, df_perf

In [9]:
df_acq, df_perf = load_dataset()

In [ ]:
df_acq.head()